1、余弦相似度

2、曼哈顿相似度

3、n元语法相似度计算

4、jaccard相似性

5、 masi距离

6、编辑距离

## 1 利用doc2vec计算余弦相似度

In [51]:
import logging
import random
import numpy as np
import pandas as pd
from gensim.models import doc2vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [52]:
# 读取数据
dataset=pd.read_csv('data/dataset.csv',sep='\t')
dataset.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [53]:
# 构建doc2vc的训练语料
corpus=[]
for index,text in enumerate(dataset.review):
    label=str(index)
    corpus.append(doc2vec.TaggedDocument(text.split(),[label]))
corpus  
    
# 训练doc2vec
logging.info("Building Doc2Vec vocabulary")
d2v=doc2vec.Doc2Vec(min_count=1,window=5,vector_size=100,
                    workers=5,alpha=0.025,min_alpha=0.00025,dm=1)

d2v.build_vocab(corpus)
logging.info("Training Doc2Vec model")
d2v.train(corpus, total_examples=d2v.corpus_count, epochs=d2v.epochs)
logging.info("Saving trained Doc2Vec model")
d2v.save("d2v.model")

2019-06-13 23:18:24,558 : INFO : Building Doc2Vec vocabulary
2019-06-13 23:18:24,560 : INFO : collecting all words and their counts
2019-06-13 23:18:24,563 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-06-13 23:18:25,699 : INFO : PROGRESS: at example #10000, processed 2354776 words (2076264/s), 161418 word types, 10000 tags
2019-06-13 23:18:27,181 : INFO : PROGRESS: at example #20000, processed 4686255 words (1574970/s), 249262 word types, 20000 tags
2019-06-13 23:18:27,756 : INFO : collected 286719 word types and 25000 unique tags from a corpus of 25000 examples and 5844691 words
2019-06-13 23:18:27,759 : INFO : Loading a fresh vocabulary
2019-06-13 23:18:35,931 : INFO : effective_min_count=1 retains 286719 unique words (100% of original 286719, drops 0)
2019-06-13 23:18:35,933 : INFO : effective_min_count=1 leaves 5844691 word corpus (100% of original 5844691, drops 0)
2019-06-13 23:18:38,027 : INFO : deleting the raw counts dictionary of 286719

2019-06-13 23:19:32,881 : INFO : EPOCH 4 - PROGRESS: at 11.20% examples, 255166 words/s, in_qsize 9, out_qsize 0
2019-06-13 23:19:33,893 : INFO : EPOCH 4 - PROGRESS: at 16.56% examples, 255039 words/s, in_qsize 9, out_qsize 0
2019-06-13 23:19:34,991 : INFO : EPOCH 4 - PROGRESS: at 22.46% examples, 255245 words/s, in_qsize 9, out_qsize 0
2019-06-13 23:19:36,011 : INFO : EPOCH 4 - PROGRESS: at 30.54% examples, 277095 words/s, in_qsize 9, out_qsize 0
2019-06-13 23:19:37,045 : INFO : EPOCH 4 - PROGRESS: at 37.69% examples, 285285 words/s, in_qsize 10, out_qsize 0
2019-06-13 23:19:38,096 : INFO : EPOCH 4 - PROGRESS: at 45.51% examples, 293569 words/s, in_qsize 10, out_qsize 0
2019-06-13 23:19:39,112 : INFO : EPOCH 4 - PROGRESS: at 51.46% examples, 291396 words/s, in_qsize 9, out_qsize 0
2019-06-13 23:19:40,116 : INFO : EPOCH 4 - PROGRESS: at 60.66% examples, 306189 words/s, in_qsize 9, out_qsize 0
2019-06-13 23:19:41,124 : INFO : EPOCH 4 - PROGRESS: at 67.49% examples, 306626 words/s, in_qs

In [54]:
# 输出doc2vec
dataset.review[5]

'I dont know why people think this is such a bad movie. Its got a pretty good plot, some good action, and the change of location for Harry does not hurt either. Sure some of its offensive and gratuitous but this is not the only movie like that. Eastwood is in good form as Dirty Harry, and I liked Pat Hingle in this movie as the small town cop. If you liked DIRTY HARRY, then you should see this one, its a lot better than THE DEAD POOL. 4/5'

In [55]:
x1='I dont know why people think this is such a bad movie. '
inferred_vector1 = d2v.infer_vector(x1.split(),steps=100, alpha=0.025)

x2='I dont know why people think this is such a bad movie. '
inferred_vector2 = d2v.infer_vector(x1.split(),steps=100, alpha=0.025)
inferred_vector1,inferred_vector2

(array([ 1.1383786 , -0.46784815,  0.06375925,  0.28962016, -0.24800839,
        -0.06971131,  0.59335   ,  0.4376741 , -0.19636081, -0.58239317,
         0.45597535, -0.18986915, -0.33574727, -0.22845191,  0.07775857,
        -0.2017226 , -0.0140759 , -0.3009961 ,  0.09014519,  0.41243365,
        -0.3334271 , -0.18992965, -0.7009654 ,  0.09367395, -0.31657284,
        -0.0486575 ,  0.58614874,  0.06907474, -0.2246884 , -0.2485166 ,
         0.47919285, -0.49073938,  0.194364  ,  0.37193924,  0.5740015 ,
        -0.04464248,  0.1534153 , -0.02575611, -0.54227084,  0.9011591 ,
        -0.57240605, -0.15567659,  0.17084531, -0.27343234, -0.40673628,
        -0.2340103 ,  0.08737222, -0.0491593 ,  0.1989162 ,  0.00204547,
         0.12318104, -0.8986703 , -0.11936127, -0.42431563,  0.01699058,
         0.41401187, -0.37953588, -0.3621248 , -0.7772428 ,  0.12070463,
         0.4678501 ,  0.10152269,  0.10237753, -0.34688845, -0.5331601 ,
         0.8923948 , -0.30411497, -0.29061875, -0.0

In [56]:
from nltk.cluster.util import cosine_distance
cosine_distance(inferred_vector1,inferred_vector2)

0.0119474694699786

![](https://images2018.cnblogs.com/blog/1182656/201803/1182656-20180301160937850-1829406764.png)

In [75]:
def cosine_similarity(vector1,vector2):
    return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*(np.linalg.norm(vector2)))
cosine_similarity(inferred_vector1, inferred_vector2)

0.9880526

## 2 基于曼哈顿空间距离计算两个字符串语义空间表示相似度计算


感觉不靠谱

In [76]:
def exponent_neg_manhattan_distance(sent_left, sent_right):
    '''基于曼哈顿空间距离计算两个字符串语义空间表示相似度计算'''
    return np.exp(-np.sum(np.abs(sent_left - sent_right)))

In [77]:
exponent_neg_manhattan_distance(inferred_vector1, inferred_vector2)

0.0049027507

## 3 n元语法相似度计算

In [60]:
import nltk
#这里展示2元语法
text1 = 'Chief Executive Officer'

#bigram考虑匹配开头和结束，所有使用pad_right和pad_left
ceo_bigrams = nltk.bigrams(text1.split(),pad_right=True,pad_left=True)
print(list(ceo_bigrams))

[(None, 'Chief'), ('Chief', 'Executive'), ('Executive', 'Officer'), ('Officer', None)]


In [61]:
import nltk

#这里展示2元语法
def bigram_distance(text1, text2):
    #bigram考虑匹配开头和结束，所以使用pad_right和pad_left
    text1_bigrams = nltk.bigrams(text1.split(),pad_right=True,pad_left=True)
    text2_bigrams = nltk.bigrams(text2.split(), pad_right=True, pad_left=True)
    #交集的长度
    distance = len(set(text1_bigrams).intersection(set(text2_bigrams)))
    return distance
text1 = 'Chief Executive Officer is manager'
text2 = 'Chief Technology Officer is technology manager'
print(bigram_distance(text1, text2)) #相似度为3



3


## 4 jaccard相似性
jaccard距离度量的两个集合的相似度，它是由 （集合1交集合2）/（结合1交结合2）计算而来的。



In [62]:
from nltk.metrics.distance import jaccard_distance

#这里我们以单个的字符代表文本
set1 = set(['a','b','c','d','a'])
set2 = set(['a','b','e','g','a'])

print(jaccard_distance(set1, set2))

0.6666666666666666


## 5 masi距离

masi距离度量是jaccard相似度的加权版本，当集合之间存在部分重叠时，通过调整得分来生成小于jaccard距离值。


In [63]:
from nltk.metrics.distance import jaccard_distance,masi_distance

#这里我们以单个的字符代表文本
set1 = set(['a','b','c','d','a'])
set2 = set(['a','b','e','g','a'])

print(jaccard_distance(set1, set2))
print(masi_distance(set1, set2))



0.6666666666666666
0.89


## 6 编辑距离

编辑距离，又称为Levenshtein距离，是用于计算一个字符串转换为另一个字符串时，插入、删除和替换的次数。例如，将'dad'转换为'bad'需要一次替换操作，编辑距离为1。

In [64]:
from nltk.metrics.distance import edit_distance

str1 = 'bad'
str2 = 'dad'
print(edit_distance(str1, str2))



1


参考：

数据集来自：https://github.com/ibrahimsharaf/doc2vec

https://github.com/iamxiaomu/doc2vec/blob/master/doc2vct.py

https://www.cnblogs.com/bymo/p/8489037.html

https://blog.csdn.net/u011897301/article/details/79083154

https://www.jb51.net/article/136217.htm